In [2]:
import os
import sys

# Get the root directory of the repository
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..')) # Assuming your notebook is in a subdirectory

# Add the root directory to Python's search path
if root_dir not in sys.path:
    sys.path.append(root_dir)

# Now you can import the function from encoding.py
from encoding import prefix_bin

# ... rest of your code ...

In [3]:
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import utils
import sliding_window

import warnings

warnings.filterwarnings("ignore")
import xgboost as xgb

import datetime, time
import importlib
importlib.reload(sliding_window)

<module 'sliding_window' from 'c:\\Users\\rexbr\\Documents\\Universität\\Master\\PM\\online_ppm_stability_HU-PM\\sliding_window.py'>

In [4]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [5]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [6]:
import os
print(os.getcwd())

c:\Users\rexbr\Documents\Universität\Master\PM\online_ppm_stability_HU-PM\expereiment


In [7]:
import json
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Correct path to the JSON file
# CHANGE HERE TO YOUR PATH
file_path = r'C:\Users\rexbr\Documents\Universität\Master\PM\online_ppm_stability_HU-PM\dataset_parameters.json'

# Load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

print(data)

{'bpic15': {'key_pair': {}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 45}, 'bpic17': {'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}, 'iro5k': {'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}, 'synthetic_log_b': {'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}, 'synthetic_log_bc1': {'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}, 'synthetic_log_bc1c2': {'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}, 'synthetic_log_bc2': {'key_pair': {'Case ID': 'caseid', '

In [17]:
'''
Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
and obtain feature matrix to transform future events
'''

# Experiment settings
dataset_label = 'bpic17'
true_to_1 = {'True': 1, 'False': 0}

# Invoke parameters for dataset
with open(file_path,'r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    performance_measure_type = 'ROCAUC'

    dataset_loc = r'C:\Users\rexbr\Documents\Universität\Master\PM\online_ppm_stability_HU-PM\DATA\logs' + '\\'+ dataset_label +'.csv'
    try:
        os.makedirs('./result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'xgb'
    if classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01, verbosity =0)

    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    casecount = 0
    rowcounter = 0
    running_case = 0
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    
    # Sliding window for training setting
    window_size = 100
    test_size = 30
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier, 0]
    
    dataset = stream.iter_csv(
            dataset_loc
            )
    totallength = len(list(dataset))


    dataset = stream.iter_csv(
                dataset_loc,
                target='outcome'
                )
    enctype = 'Index-base'
    
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0

    for x,y in dataset:
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(maximum_prefix)]=set()

                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    x = streaming_db[caseid][maximum_prefix-1]
                    training_windows.update_window(x)
                    update_test_cases(streaming_db[caseid], test_cases)

                    if training_windows.retraining_count == test_size:
                        x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                        for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                        feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values
                        y_training = [i.true_label for i in training_windows.container]
                        # print(y_training)
                        y_training_encoded = le.fit_transform(y_training)
                        # print(y_training_encoded)
                        training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training_encoded)
                        training_models['prefix_%s'%(maximum_prefix)][1] +=1
                        training_windows.reset_retraining_count()

                        '''
                        Evaluate current model with test cases
                        '''
                        model_update_count = training_models['prefix_%s'%(maximum_prefix)][1]
                        prediction_result[maximum_prefix][model_update_count] = {}
                        y_truelist = []
                        y_predlist = []
                        for case in test_cases:
                            if len(case) >= maximum_prefix:
                                x = case[maximum_prefix-1]
                                model = training_models['prefix_%s'%(x.prefix_length)][0]
                                current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                                current_event = pd.Series(current_event).to_frame().T
                                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba(current_event)
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                        prediction_result[maximum_prefix][model_update_count]['y_true'] = y_truelist
                        prediction_result[maximum_prefix][model_update_count]['y_pred'] = y_predlist
                        if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                            cdhappend[maximum_prefix] = model_update_count

    # print("y truelist", y_truelist, "length", len(y_truelist))
    # print("y predlist", y_predlist, "length", len(y_predlist))
    for evaluation_method in ['Accuracy', 'F1', 'ROCAUC']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob_ = prediction_result[prefix][update]['y_pred']
            prediction_class = training_models['prefix_%s'%(maximum_prefix)][0].classes_
            if len(prediction_class) == 1: prediction_class = ['False', 'True']
            y_pred_prob = []
            for i in y_pred_prob_:
                # print("i:", i)
                y_pred_prob.append({prediction_class[0]: i[0][0], prediction_class[1]: i[0][1]})
            # print("y_pred_prob", y_pred_prob)
            y_pred = [max(x, key=x.get) for x in y_pred_prob]
            print("y_pred", y_pred)
            print("y_true", y_true)
            if maximum_prefix == 15:
                # for some reason, the labels in y_pred are "True" and "False" instead of 1 and 0, while all the previous
                # predictions are 1 and 0. This is a fix to make the labels consistent
                y_pred = [true_to_1[y] for y in y_pred]
                print("y_pred", y_pred)
            # Fit and transform y_true and y_pred
            y_true_encoded = le.fit_transform(y_true)
            # Ensure y_pred is a numpy array of strings
            # Filter out unseen labels
            seen_labels = set(le.classes_)
            y_pred_filtered = [label for label in y_pred if label in seen_labels]

            print("y_true_encoded", y_true_encoded)
            # print("y_pred_encoded", y_pred_encoded)
            update_count.append(update)

            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true_encoded, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true_encoded, y_pred, average='macro')
                # warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                print("y_pred_prob", y_pred_prob)
                if maximum_prefix == 15:
                    # for prefix 15, the y_pred_prob is a list of string values again, so we need to access the values with 'True' and 'False' again
                    y_pred_prob = np.array([[x['False'], x['True']] for x in y_pred_prob])
                else:
                    y_pred_prob = np.array([[x[0], x[1]] for x in y_pred_prob]) # np.array([[x['False'], x['True']] for x in y_pred_prob])
                # warnings.filterwarnings("ignore")
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:, 1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')
        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)

        if os.path.exists('./img/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('./img/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))
        if os.path.exists('./result/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('./result/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))


        with open('./result/%s/%s/Model updates/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('./img/%s/%s/Model updates/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
15
y_pred ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False']
y_true ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False']
y_pred [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_true_encoded [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_pred ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False',

In [ ]:
'''
Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
and obtain feature matrix to transform future events
'''

# Experiment settings
dataset_label = 'bpic17'

# Invoke parameters for dataset
with open(file_path,'r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    performance_measure_type = 'Accuracy'

    dataset_loc = r'C:\Users\rexbr\Documents\Universität\Master\PM\online_ppm_stability_HU-PM\DATA\logs' + '\\'+ dataset_label +'.csv'
    # dataset_loc = './DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('./result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'xgb'
    if classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01, verbosity =0)

    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    casecount = 0
    rowcounter = 0
    running_case = 0
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    
    # Sliding window for training setting
    window_size = 100
    test_size = 30
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier, 0]
    
    dataset = stream.iter_csv(
            dataset_loc
            )
    totallength = len(list(dataset))


    dataset = stream.iter_csv(
                dataset_loc,
                target='outcome'
                )
    enctype = 'Index-base'
    
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0

    for x,y in dataset:
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(maximum_prefix)]=set()

                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    x = streaming_db[caseid][maximum_prefix-1]
                    training_windows.update_window(x)
                    update_test_cases(streaming_db[caseid], test_cases)

                    if training_windows.retraining_count == test_size:
                        x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                        for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                        feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values
                        y_training = [i.true_label for i in training_windows.container]
                        y_training_encoded = le.fit_transform(y_training)
                        # training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training)
                        training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training_encoded)
                        training_models['prefix_%s'%(maximum_prefix)][1] +=1
                        training_windows.reset_retraining_count()

                        '''
                        Evaluate current model with test cases
                        '''
                        prediction_result[maximum_prefix][casecount] = {}
                        y_truelist = []
                        y_predlist = []
                        for case in test_cases:
                            if len(case) >= maximum_prefix:
                                x = case[maximum_prefix-1]
                                model = training_models['prefix_%s'%(x.prefix_length)][0]
                                current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                                current_event = pd.Series(current_event).to_frame().T
                                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba(current_event)
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                        prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                        prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
                        if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                            cdhappend[maximum_prefix] = casecount
    for evaluation_method in ['Accuracy', 'F1', 'ROCAUC']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob_ = prediction_result[prefix][update]['y_pred']
            prediction_class = training_models['prefix_%s'%(maximum_prefix)][0].classes_
            if len(prediction_class) == 1: prediction_class = ['False', 'True']
            elif len(prediction_class) == 2: prediction_class = ['False', 'True']
            y_pred_prob = []
            for i in y_pred_prob_:
                y_pred_prob.append({prediction_class[0]: i[0][0], prediction_class[1]: i[0][1]})
            y_pred = [max(x, key=x.get) for x in y_pred_prob]
            update_count.append(update)

            # DEBUGGING
            print("y_pred", y_pred)
            print("y_true", y_true)
            #if maximum_prefix == 15:
                # for some reason, the labels in y_pred are "True" and "False" instead of 1 and 0, while all the previous
                # predictions are 1 and 0. This is a fix to make the labels consistent
             #   y_pred = [true_to_1[y] for y in y_pred]
              #  print("y_pred", y_pred)
            # Fit and transform y_true and y_pred
            y_true_encoded = le.fit_transform(y_true)
            # Ensure y_pred is a numpy array of strings
            # Filter out unseen labels
            seen_labels = set(le.classes_)
            y_pred_filtered = [label for label in y_pred if label in seen_labels]
            # ========================
            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true_encoded, y_pred)
                # value = accuracy_score(y_true, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true, y_pred, average='macro')
                # value = f1_score(y_true, y_pred, average='macro')
        #             warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                y_pred_prob = np.array([[x['False'], x['True']] for x in y_pred_prob])
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:, 1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')
        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)

        if os.path.exists('./img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('./img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))
        if os.path.exists('./result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('./result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))


        with open('./result/%s/%s/Finished cases/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('./img/%s/%s/Finished cases/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
15
y_pred ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False']
y_true ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False']
y_pred ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'Fa

In [7]:
prediction_result

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}